In [1]:
import sys,os, pickle
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from transformers import BertForMaskedLM
from torch.utils.data import DataLoader, Dataset

from transformers import AdamW
from tqdm import tqdm  # for our progress bar



with (open('DFGDATA.pkl' , "rb")) as openfile:
    all_input_list , all_label_list = pickle.load(openfile)


# input_text = ["I love apples.", "She hates oranges.", "I enjoy reading books.", "He likes playing football."]

# labels_task1 = [0, 1,0,1]  # Example task 1 labels
# labels_task2 = [2, 3,4,5]  # Example task 2 labels

input_text = all_input_list[:8]
labels_dfg = all_label_list[:8]


/home/nahid/anaconda3/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer  = BertTokenizer.from_pretrained("./binary-tokenizer")
VOCAB_SIZE = tokenizer.vocab_size

print('VOCAB SIZE : ', (tokenizer.vocab_size))


VOCAB SIZE :  7718


In [3]:
# from transformers import DataCollatorForLanguageModeling

# tokenizer.eos_token = "<EOI>"

# tokenizer.pad_token = tokenizer.eos_token
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [4]:
# data_collator(input_text_task1)

In [5]:
#TODO fix inputs = tokenizer(history, next_instruction, return_tensors='pt', 
#                    max_length=64, truncation=True, padding='max_length')

# Tokenize input text
inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors='pt',)
# encoded_inputs_task2 = tokenizer(input_text_task2, padding=True, truncation=True, return_tensors='pt')

inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [6]:
#MLM
inputs['labels'] = inputs.input_ids.detach().clone()
inputs.labels[0]

# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
        (inputs.input_ids != 102) * (inputs.input_ids != 0)

#TODO FIX 101, 102

# mask_arr[0]

In [7]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )
# selection[0]

In [8]:
# inputs.input_ids

In [9]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [10]:
# inputs.input_ids

In [11]:
print(inputs.keys())
print(inputs.token_type_ids)
inputs.token_type_ids[0][0] =1
print(inputs.token_type_ids)
#TODO set token type ids

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


In [12]:
print(VOCAB_SIZE)

7718


In [13]:

# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Define the multitask BERT model

hidden_size = BertModel.from_pretrained('bert-base-uncased').config.hidden_size
# vocab_size = bert_model.config.vocab_size
class MultitaskBert(nn.Module):
    def __init__(self, num_classes_task1, num_classes_task2):
        super(MultitaskBert, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased') #BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(  hidden_size , VOCAB_SIZE)
        self.task2_classifier = nn.Linear(hidden_size, 2) #todo was 768 num_classes_task2



    def forward(self, t1_input_ids, t1_attention_mask , t2_input_ids, t2_attention_mask):
        bert_outputs = self.bert(t1_input_ids, attention_mask=t1_attention_mask)
        
#         logits_mlm = bert_outputs.last_hidden_state
        hidden_states = bert_outputs.last_hidden_state
        logits_mlm = self.fc(hidden_states)
        
        bert_pooled_output = bert_outputs.pooler_output
        logits_task2 = self.task2_classifier(bert_pooled_output)
        
        
#         t2_outputs = self.bert(t2_input_ids, attention_mask=t2_attention_mask)
#         t2_pooled_output = t2_outputs.pooler_output
#         t2_outputs = self.task2_classifier(t1_input_ids, attention_mask=t1_attention_mask)#TODO t1/t2 fix

        return logits_mlm,   logits_task2




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
# Create model instance
model = MultitaskBert(num_classes_task1=VOCAB_SIZE, num_classes_task2=2)  # Example number of classes for each task

# initialize optimizer
optim = AdamW( model.parameters() , lr=5e-6)
criterion = nn.CrossEntropyLoss()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:

# print(inputs['labels'])
# print(torch.squeeze(inputs['labels']))

In [16]:
# Forward pass
optim.zero_grad()
# logits_task1, logits_task2 , t1_outputs,t2_outputs = model(inputs['input_ids'], inputs['attention_mask'], inputs['input_ids'], inputs['attention_mask'])

logits_task1,   logits_task2 = model(inputs['input_ids'], inputs['attention_mask'], inputs['input_ids'], inputs['attention_mask'])
print(logits_task1.shape, logits_task2.shape)

torch.Size([8, 512, 7718]) torch.Size([8, 2])


In [17]:
print(logits_task1.shape , inputs['labels'].shape)

torch.Size([8, 512, 7718]) torch.Size([8, 512])


In [18]:
logits_task1.view(-1, VOCAB_SIZE).shape ,inputs['labels'].view(-1).shape

(torch.Size([4096, 7718]), torch.Size([4096]))

In [19]:
loss_task1 = criterion(logits_task1.view(-1, VOCAB_SIZE), inputs['labels'].view(-1))
predictions_task1 = torch.argmax(logits_task1, dim=2)

In [20]:
labels_dfg =  torch.Tensor(labels_dfg)
labels_dfg = labels_dfg.type(torch.LongTensor)
labels_dfg.shape
logits_task2.shape

torch.Size([8, 2])

In [21]:
loss_task2 = criterion(logits_task2, labels_dfg)
predictions_task2 = torch.argmax(logits_task2, dim=1)
predictions_task2

tensor([1, 1, 1, 1, 1, 1, 1, 1])

In [22]:
(loss_task1+loss_task2).backward()
optim.step()